# Semester 1 Project Submission

Please fill out:
* Student name: Cristian González Ramírez
* Instructor name: Nick McCarty & Julian Ward

## This notebook tries to answer to the following question:

How are the production budgets for movies of different genres in comparison to their income? Would it be profitable?

For this question specifically, we are interested on knowing the Production Costs, the Total Gross (both domestic and foreign) and the genres. We find this info in the following dataframes and tables:`movie_budgets` and `SQL im.db"`

### Importing libraries
We will start by adding the basic libraries to our notebook, so it runs nice and smoothly.

In [434]:
# Import all libraries the project will requiere

#For retreiving data
import pandas as pd
import sqlite3
import zipfile

#For analyzing data
from scipy import stats

#For plotting and visulaizations
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

### Retreiving general information
Then, we insert the data we will be working with.

In [478]:
#Retrieving data to work with
#movie_gross_data = pd.read_csv("zippedData/bom.movie_gross.csv.gz")
#movies           = pd.read_csv("zippedData/tmdb.movies.csv.gz")
movie_budgets    = pd.read_csv("zippedData/tn.movie_budgets.csv.gz")

#movie_info_data  = pd.read_csv("zippedData/rt.movie_info.tsv.gz", sep="\t")

In [471]:
#Oppening SQLite3 zip
with zipfile.ZipFile("zippedData/im.db.zip", "r") as zObject:
        zObject.extractall()

conn = sqlite3.connect("im.db")
cur = conn.cursor()

### Cleaning Movie Budgets data

Before starting any anylisis at all, we have to be sure that the data we are dealing with is ready to be manipulated. In order to accopmlish it, we musgt clean the data from all the tables we are using. First we will start with `movie_budget`

In [468]:
movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [438]:
movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


As we see all of the values are a `Not-Null`, so most of the job is done; even so, a few considerations must be taken.

Here we check if there is any movie appearing more than once.

In [441]:
print(len(movie_budgets.movie.unique()), movie_budgets.shape[0])

5698 5782


Since these two numbers differ, we must take a look into it and remove any duplicates if any.

In [444]:
movie_budgets.movie.value_counts()

King Kong            3
Halloween            3
Home                 3
Friday the 13th      2
Heist                2
                    ..
The Equalizer 2      1
Romeo Is Bleeding    1
Playing for Keeps    1
Trolls               1
Jackie               1
Name: movie, Length: 5698, dtype: int64

In fact, we have a bunch of repeating movies. We have to deal with them, so they only appear once.

In [456]:
movie_budgets.drop_duplicates(subset=["movie"], inplace = True)

In [457]:
print(len(movie_budgets.movie.unique()), movie_budgets.shape[0])

5698 5698


Now both values are similiar, which let us know that we got rid of duplicates perfectly.

Runnig the `.info()` command above, we also have found that `production_budget`, `domestic_gross` and `worldwide_gross` are object types, when we need them to be a numeric value, to perform further comparisons. So we have to take care of that issue too.

In [479]:
#Getting rid off commas so it can be casted
movie_budgets.production_budget.replace(",","",regex=True, inplace=True)
movie_budgets.domestic_gross.replace(",","",regex=True, inplace=True)
movie_budgets.worldwide_gross.replace(",","",regex=True, inplace=True)

#Getting rid off $
movie_budgets["production_budget"] = movie_budgets.production_budget.str.strip("$")
movie_budgets["domestic_gross"] = movie_budgets.domestic_gross.str.strip("$")
movie_budgets["worldwide_gross"] = movie_budgets.worldwide_gross.str.strip("$")

In [480]:
#Casting foreign_gross as a float
movie_budgets["production_budget"] = movie_budgets.production_budget.astype(float)
movie_budgets["domestic_gross"] = movie_budgets.domestic_gross.astype(float)
movie_budgets["worldwide_gross"] = movie_budgets.worldwide_gross.astype(float)

In [481]:
movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,425000000.0,760507625.0,2.776345e+09
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09
2,3,"Jun 7, 2019",Dark Phoenix,350000000.0,42762350.0,1.497624e+08
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000.0,459005868.0,1.403014e+09
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000.0,620181382.0,1.316722e+09


It could be useful to have a `total_gross` and `profit_or_loss` column for easier analysis later.

In [482]:
movie_budgets["total_gross"] = movie_budgets.domestic_gross + movie_budgets.worldwide_gross
movie_budgets["profit_or_loss"] = movie_budgets.total_gross - movie_budgets.production_budget

In [483]:
movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,total_gross,profit_or_loss
0,1,"Dec 18, 2009",Avatar,425000000.0,760507625.0,2.776345e+09,3.536853e+09,3.111853e+09
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09,1.286728e+09,8.761278e+08
2,3,"Jun 7, 2019",Dark Phoenix,350000000.0,42762350.0,1.497624e+08,1.925247e+08,-1.574753e+08
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000.0,459005868.0,1.403014e+09,1.862020e+09,1.531420e+09
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000.0,620181382.0,1.316722e+09,1.936903e+09,1.619903e+09


### Retreiving Movie ratings data
Now we get the info from the relevant SQL table for the analysis.

In [411]:
#Obtain relevant information from SQL database
query = '''
    SELECT
        movie_id AS "ID",
        primary_title AS "Title",
        original_title AS "Original Title",
        genres AS "Genres",
        averagerating AS "Rating",
        numvotes AS "Number of Votes"
    FROM movie_basics
        JOIN movie_ratings
        USING (movie_id)
'''

#Assign the query to a pandas dataframe
movie_rating = pd.read_sql(query,conn)

In [412]:
movie_rating.head(10)

,ID,Title,Original Title,Genres,Rating,Number of Votes
0,tt0063540,Sunghursh,Sunghursh,"Action,Crime,Drama",7.0,77
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,"Biography,Drama",7.2,43
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,Drama,6.9,4517
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,"Comedy,Drama",6.1,13
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,"Comedy,Drama,Fantasy",6.5,119
5,tt0112502,Bigfoot,Bigfoot,"Horror,Thriller",4.1,32
6,tt0137204,Joe Finds Grace,Joe Finds Grace,"Adventure,Animation,Comedy",8.1,263
7,tt0146592,Pál Adrienn,Pál Adrienn,Drama,6.8,451
8,tt0154039,So Much for Justice!,Oda az igazság,History,4.6,64
9,tt0159369,Cooper and Hemingway: The True Gen,Cooper and Hemingway: The True Gen,Documentary,7.6,53


### Cleaning SQL data

In [413]:
movie_rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               73856 non-null  object 
 1   Title            73856 non-null  object 
 2   Original Title   73856 non-null  object 
 3   Genres           73052 non-null  object 
 4   Rating           73856 non-null  float64
 5   Number of Votes  73856 non-null  int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 3.4+ MB


As we can see, most of the data we obtain is complete, but there may still be some values that could mess up our analysis, so we better clean this data as well!

In [414]:
movie_rating.Title.value_counts()

The Return              11
Broken                  10
Together                 9
Lucky                    9
Homecoming               9
                        ..
Peculiar Pets            1
Pour l'amour de Dieu     1
After Party              1
Red Cow                  1
Shilo                    1
Name: Title, Length: 69993, dtype: int64

In [415]:
movie_rating.drop_duplicates(subset=["Title"], inplace=True)

In [416]:
movie_rating.Title.value_counts()

Violentia                 1
Searching                 1
Money and Medicine        1
The Funhouse Massacre     1
Senza lasciare traccia    1
                         ..
These Birds Walk          1
Trumpocalypse Now!        1
Night School              1
Deadfall                  1
Shilo                     1
Name: Title, Length: 69993, dtype: int64

In [417]:
movie_rating["Original Title"].value_counts()

Missed Connections          3
Mundo Cão                   2
Iris                        2
Ra                          2
Snatched                    2
                           ..
Auf den zweiten Blick       1
Angelus Hiroshimae          1
Molly's Girl                1
The Melancholy Fantastic    1
Shilo                       1
Name: Original Title, Length: 69820, dtype: int64

In [418]:
movie_rating.drop_duplicates(subset=["Original Title"], inplace = True)

In [419]:
movie_rating["Original Title"].value_counts()

Violentia                       1
C'est tout pour moi             1
And Who Taught You to Drive?    1
Dhairyam                        1
Satri lek tob lok taek          1
                               ..
Qian Xuesen                     1
Die Präsenz                     1
Life of a King                  1
El Teniente Amado               1
Shilo                           1
Name: Original Title, Length: 69820, dtype: int64

In [420]:
movie_rating.Rating.unique()

array([ 7. ,  7.2,  6.9,  6.1,  6.5,  4.1,  8.1,  6.8,  4.6,  7.6,  7.5,
        7.8,  4. ,  8.8,  1.9,  6.7,  6.6,  5.9,  7.1,  7.4,  3.9,  5.5,
        5.1,  7.9,  6.2,  6.3,  5. ,  6. ,  5.6,  7.3,  4.9,  6.4,  3.3,
        5.7,  8.6,  4.8,  4.5,  4.2,  8.7,  4.7,  5.2,  8.3,  9.1,  3.4,
        5.8,  5.4,  8.5,  3.2,  3.6,  8. ,  3.8,  7.7,  4.3,  3. ,  5.3,
        3.7,  3.5,  9.3,  8.9,  9. ,  2.3,  8.2,  2.7,  2.8,  1.7,  4.4,
        2.2,  9.4,  8.4,  9.2,  9.7,  3.1, 10. ,  9.8,  2.6,  2.5,  1.4,
        2.1,  2.4,  2.9,  1.8,  1.6,  2. ,  1. ,  1.2,  1.5,  1.3,  1.1,
        9.5,  9.6,  9.9])

We can see there is no apparent problem in the Rating column

In [421]:
movie_rating.ID.value_counts().sort_values(ascending = True)

tt8428316    1
tt2354215    1
tt7054868    1
tt1334455    1
tt7397958    1
            ..
tt1795046    1
tt6096958    1
tt6202264    1
tt8998610    1
tt1619814    1
Name: ID, Length: 69820, dtype: int64

Nor there is any in the ID column. 

Most of the cleaning of this Dataframe is done. The next step to do is to decide how to procede with the missing values we saw in the Genres column. There are around 800 missing values, and dropping those columns won't represent as much of a loss, so we will proceed with this approach.

In [422]:
movie_ratinge = movie_rating[movie_rating.Genres.str.contains("None") == False]

In [423]:
movie_rating.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69820 entries, 0 to 73854
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               69820 non-null  object 
 1   Title            69820 non-null  object 
 2   Original Title   69820 non-null  object 
 3   Genres           69059 non-null  object 
 4   Rating           69820 non-null  float64
 5   Number of Votes  69820 non-null  int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 3.7+ MB


We can see we have a dataframe without any null value or duplicates. We can consider this dataframe to be clean!

Nevertheless, before moving forward, I will consider an extra step for singificance in later analysis; which is getting the percentage of rating base on the number of votes. Where we will use the greatest `Number of votes` as reference to all the movies, i.e. the movie with its `Number of voters` value equal to `max_voters` will have a significance of `1`.

In [424]:
max_voters = movie_rating["Number of Votes"].max()
movie_rating["Significance"] = (movie_rating["Number of Votes"]/max_voters)

In [425]:
movie_rating.head()

,ID,Title,Original Title,Genres,Rating,Number of Votes,Significance
0,tt0063540,Sunghursh,Sunghursh,"Action,Crime,Drama",7.0,77,0.000042
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,"Biography,Drama",7.2,43,0.000023
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,Drama,6.9,4517,0.002453
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,"Comedy,Drama",6.1,13,0.000007
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,"Comedy,Drama,Fantasy",6.5,119,0.000065


In [426]:
movie_rating.loc[(movie_rating["Number of Votes"]) == max_voters]

,ID,Title,Original Title,Genres,Rating,Number of Votes,Significance
2387,tt1375666,Inception,Inception,"Action,Adventure,Sci-Fi",8.8,1841066,1.0


### Joining tables
Now, we shall proceed with joining the `movie_rating` table with the `movie_general_gross` one to proceed with the analysis.

In [427]:
movie_general_gross.head()

,title,studio,domestic_gross,foreign_gross,year,total_gross
727,Marvel's The Avengers,BV,623400000.0,895500000.0,2012,1.518900e+09
1875,Avengers: Age of Ultron,BV,459000000.0,946400000.0,2015,1.405400e+09
3080,Black Panther,BV,700100000.0,646900000.0,2018,1.347000e+09
328,Harry Potter and the Deathly Hallows Part 2,WB,381000000.0,960500000.0,2011,1.341500e+09
2758,Star Wars: The Last Jedi,BV,620200000.0,712400000.0,2017,1.332600e+09


In [428]:
#Create a new Dataframe
gross_and_rating = pd.merge(movie_general_gross, movie_rating, how="inner", left_on="title", right_on="Title")

In [429]:
gross_and_rating.head()

,title,studio,domestic_gross,foreign_gross,year,total_gross,ID,Title,Original Title,Genres,Rating,Number of Votes,Significance
0,Avengers: Age of Ultron,BV,459000000.0,946400000.0,2015,1.405400e+09,tt2395427,Avengers: Age of Ultron,Avengers: Age of Ultron,"Action,Adventure,Sci-Fi",7.3,665594,0.361526
1,Black Panther,BV,700100000.0,646900000.0,2018,1.347000e+09,tt1825683,Black Panther,Black Panther,"Action,Adventure,Sci-Fi",7.3,516148,0.280353
2,Star Wars: The Last Jedi,BV,620200000.0,712400000.0,2017,1.332600e+09,tt2527336,Star Wars: The Last Jedi,Star Wars: Episode VIII - The Last Jedi,"Action,Adventure,Fantasy",7.1,462903,0.251432
3,Jurassic World: Fallen Kingdom,Uni.,417700000.0,891800000.0,2018,1.309500e+09,tt4881806,Jurassic World: Fallen Kingdom,Jurassic World: Fallen Kingdom,"Action,Adventure,Sci-Fi",6.2,219125,0.119021
4,Frozen,BV,400700000.0,875700000.0,2013,1.276400e+09,tt1323045,Frozen,Frozen,"Adventure,Drama,Sport",6.2,62311,0.033845


In [430]:
#Deleting duplicated columns

del gross_and_rating["Title"]
del gross_and_rating["Original Title"]

In [431]:
gross_and_rating.head()

,title,studio,domestic_gross,foreign_gross,year,total_gross,ID,Genres,Rating,Number of Votes,Significance
0,Avengers: Age of Ultron,BV,459000000.0,946400000.0,2015,1.405400e+09,tt2395427,"Action,Adventure,Sci-Fi",7.3,665594,0.361526
1,Black Panther,BV,700100000.0,646900000.0,2018,1.347000e+09,tt1825683,"Action,Adventure,Sci-Fi",7.3,516148,0.280353
2,Star Wars: The Last Jedi,BV,620200000.0,712400000.0,2017,1.332600e+09,tt2527336,"Action,Adventure,Fantasy",7.1,462903,0.251432
3,Jurassic World: Fallen Kingdom,Uni.,417700000.0,891800000.0,2018,1.309500e+09,tt4881806,"Action,Adventure,Sci-Fi",6.2,219125,0.119021
4,Frozen,BV,400700000.0,875700000.0,2013,1.276400e+09,tt1323045,"Adventure,Drama,Sport",6.2,62311,0.033845


In [432]:
gross_and_rating.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1534 entries, 0 to 1533
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title            1534 non-null   object 
 1   studio           1534 non-null   object 
 2   domestic_gross   1534 non-null   float64
 3   foreign_gross    1534 non-null   float64
 4   year             1534 non-null   int64  
 5   total_gross      1534 non-null   float64
 6   ID               1534 non-null   object 
 7   Genres           1533 non-null   object 
 8   Rating           1534 non-null   float64
 9   Number of Votes  1534 non-null   int64  
 10  Significance     1534 non-null   float64
dtypes: float64(5), int64(2), object(4)
memory usage: 143.8+ KB


Finally, after getting all of our interest values together, the next step would be to jump into analysis, in which we compare the income of a movie based on its genre with the production budget it had.